## Stuff to load everything

In [1]:
import rootutils

rootutils.setup_root(".", indicator=".project-root", pythonpath=True)

import hydra
from hydra.experimental import initialize, compose

initialize(config_path="../configs", job_name="job")
config = compose(config_name="train.yaml", return_hydra_config=True)

/Users/gbg141/Documents/TopoProjectX/TopoBenchmarkX/venv_topox/lib/python3.11/site-packages/hydra/experimental/initialize.py:43: UserWarning: hydra.experimental.initialize() is no longer experimental. Use hydra.initialize()
  deprecation_warning(message=message)
/Users/gbg141/Documents/TopoProjectX/TopoBenchmarkX/venv_topox/lib/python3.11/site-packages/hydra/experimental/initialize.py:45: UserWarning: 
The version_base parameter is not specified.
Please specify a compatability version level, or None.
Will assume defaults for version 1.1
  self.delegate = real_initialize(
/Users/gbg141/Documents/TopoProjectX/TopoBenchmarkX/venv_topox/lib/python3.11/site-packages/hydra/experimental/compose.py:25: UserWarning: hydra.experimental.compose() is no longer experimental. Use hydra.compose()
  deprecation_warning(message=message)


In [2]:
import torch_geometric
import torch

In [3]:
from torch_geometric.utils import k_hop_subgraph
from torch_geometric.utils import to_undirected

In [4]:
from topobenchmarkx.data.transforms.liftings import HypergraphKHopLifting
from topobenchmarkx.data.transforms.lifted_dataset import Transform, LiftedDataset

In [5]:
data_mutag = torch_geometric.datasets.TUDataset(root="../data/graph", name="MUTAG")
list_of_data = [d for d in data_mutag]
lift = HypergraphKHopLifting(k=1)
transform = Transform(lift)
list_of_data = transform.transform(list_of_data)

# lifted_mutag = LiftedDataset(list_of_data)
print(data_mutag.get(0))
print(list_of_data[0])

ClientConnectorSSLError: Cannot connect to host www.chrsmrrs.com:443 ssl:default [[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed (_ssl.c:1002)]

In [80]:
type(data_mutag.get(0))

torch_geometric.data.data.Data

In [72]:
lift = KNearestNeighborsLifting(k=1)

n_graphs = len(data_mutag)
loader = torch_geometric.loader.DataLoader(data_mutag, batch_size=int(n_graphs / 2))

print(data_mutag)

for _ in range(3):
    for batch_idx, batch in enumerate(loader):
        batch = lift(batch, batch_idx)
        print(batch)

MUTAG(188)
DataBatch(edge_index=[2, 3810], x=[1713, 7], edge_attr=[3810, 4], y=[94], batch=[1713], ptr=[95], pos=[1713, 7], hyperedges=[1713, 1713])
DataBatch(edge_index=[2, 3632], x=[1658, 7], edge_attr=[3632, 4], y=[94], batch=[1658], ptr=[95], pos=[1658, 7], hyperedges=[1658, 1658])
DataBatch(edge_index=[2, 3810], x=[1713, 7], edge_attr=[3810, 4], y=[94], batch=[1713], ptr=[95], pos=[1713, 7], hyperedges=[1713, 1713])
DataBatch(edge_index=[2, 3632], x=[1658, 7], edge_attr=[3632, 4], y=[94], batch=[1658], ptr=[95], pos=[1658, 7], hyperedges=[1658, 1658])
DataBatch(edge_index=[2, 3810], x=[1713, 7], edge_attr=[3810, 4], y=[94], batch=[1713], ptr=[95], pos=[1713, 7], hyperedges=[1713, 1713])
DataBatch(edge_index=[2, 3632], x=[1658, 7], edge_attr=[3632, 4], y=[94], batch=[1658], ptr=[95], pos=[1658, 7], hyperedges=[1658, 1658])


In [73]:
def get_cora(cfg):
    data_dir = cfg["data_dir"]
    print(f"Loading {cfg['data_domain']} dataset name: {cfg['data_name']}")

    data = torch_geometric.datasets.Planetoid(data_dir, "Cora")
    return data

In [74]:
from torch_geometric.loader.dataloader import DataLoader

data_cora = get_cora(config.data)
n_graphs = len(data_cora)
loader = torch_geometric.loader.DataLoader(data_cora, batch_size=n_graphs)

Loading hypergraph dataset name: cora


In [75]:
for batch_idx, batch in enumerate(loader):
    batch = lift(batch, batch_idx)
    print(batch)

print(data_cora)

DataBatch(edge_index=[2, 3810], x=[1713, 7], edge_attr=[3810, 4], y=[94], batch=[1713], ptr=[95], pos=[1713, 7], hyperedges=[1713, 1713])
Cora()


In [78]:
print(batch.hyperedges.indices()[1][batch.hyperedges.indices()[0] == 0])

tensor([10, 11])


In [57]:
model = hydra.utils.instantiate(config.model)

/usr/local/lib/python3.11/site-packages/lightning/pytorch/utilities/parsing.py:198: Attribute 'backbone' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['backbone'])`.
/usr/local/lib/python3.11/site-packages/lightning/pytorch/utilities/parsing.py:198: Attribute 'readout_workaround' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['readout_workaround'])`.
/usr/local/lib/python3.11/site-packages/lightning/pytorch/utilities/parsing.py:198: Attribute 'readout' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['readout'])`.


In [59]:
results = model(data_lifted.x, data_lifted.hyperedges)

NameError: name 'data_lifted' is not defined

In [20]:
print(results)

(tensor([[1.1128, 0.0000, 0.0000,  ..., 0.0000, 1.0280, 0.0000],
        [1.1101, 0.0000, 0.0000,  ..., 0.0000, 0.8908, 0.0000],
        [1.2814, 0.0000, 0.0000,  ..., 0.0000, 1.4679, 0.0000],
        ...,
        [0.8512, 0.0000, 0.0000,  ..., 0.0000, 1.5145, 0.0000],
        [1.0093, 0.0000, 0.0000,  ..., 0.0000, 1.0170, 0.0000],
        [1.0837, 0.0000, 0.0000,  ..., 0.0000, 0.9484, 0.1635]],
       grad_fn=<MulBackward0>), tensor([[0.0000, 0.0000, 0.8117,  ..., 0.0000, 1.9955, 0.0000],
        [0.0000, 0.0000, 0.6481,  ..., 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 2.2122, 0.0000],
        ...,
        [0.0000, 0.0000, 0.6992,  ..., 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.8787,  ..., 0.0000, 1.6070, 0.0000],
        [0.0000, 0.0000, 0.8623,  ..., 0.0000, 2.3690, 0.0000]],
       grad_fn=<MulBackward0>))


In [55]:
import sys

In [58]:
print(sys.getsizeof(model))

56
